In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 44.6 MB/s 
     |████████████████████████████████| 163 kB 76.0 MB/s 
     |████████████████████████████████| 6.6 MB 60.6 MB/s 


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
# Import and evaluate each test batch using Matthew's correlation coefficient
from sklearn.metrics import accuracy_score,matthews_corrcoef

import random
import os
import io

from torch import nn
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from transformers import DistilBertTokenizer, BertConfig, AdamW, BertForSequenceClassification,get_linear_schedule_with_warmup, DistilBertForSequenceClassification

In [4]:
from tqdm import tqdm, trange,tnrange,tqdm_notebook
from torch.optim import Adam

In [5]:
# identify and specify the GPU as the device, later in training loop we will load data into device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#n_gpu = torch.cuda.device_count()
#torch.cuda.get_device_name(0)

SEED = 19

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if device == torch.device("cuda"):
    torch.cuda.manual_seed_all(SEED)

In [6]:
device

device(type='cuda')

In [7]:
data = pd.read_csv('../data/empathybalaceddata.csv')

In [8]:
data.head()

,response_post,level
0,"That's pretty vague, do you not know what you'...",0
1,idk what a Red pill means exactly but my advic...,0
2,"I think it's social anxiety , that creates par...",1
3,By any chance do you think you're in a loop. J...,1
4,depression. not sadness which is caused by som...,1


In [9]:
# array of sentences
sentences =  data.response_post.values

In [10]:
batch_size = 32
MAX_LEN = 128

In [11]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [12]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):
        self.labels = [label for label in df['level']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = MAX_LEN, truncation=True,
                                return_tensors="pt") for text in df['response_post']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [13]:
np.random.seed(112)
df_train, df_val, df_test = np.split(data.sample(frac=1, random_state=42), 
                                     [int(.8*len(data)), int(.9*len(data))])

print(len(df_train),len(df_val), len(df_test))

3370 421 422


In [14]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        
        super(BertClassifier, self).__init__()

        self.bert = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=768)
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        pooled_output = self.bert(input_ids= input_id, attention_mask=mask).logits
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [16]:
def train(model, train_data, val_data, learning_rate, epochs, adam_epsilon = 1e-8):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size)
    
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    
    criterion = nn.CrossEntropyLoss()
    #optimizer = Adam(model.parameters(), lr= learning_rate)
    optimizer = AdamW(model.parameters(), lr=learning_rate, eps=adam_epsilon, correct_bias=False)  # To reproduce BertAdam specific behavior set correct_bias=False
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)  # PyTorch scheduler

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                
                # Clip the norm of the gradients to 1.0
                # Gradient clipping is not in AdamW anymore
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                
                # Update parameters and take a step using the computed gradient
                optimizer.step()
                # Update learning rate schedule
                scheduler.step()
                # Clear the previous accumulated gradients
                optimizer.zero_grad()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')


EPOCHS = 5
model = BertClassifier()
LR = 2e-6 #1e-5
num_warmup_steps = 0
num_training_steps = len(df_train)*EPOCHS
adam_epsilon = 1e-8
train(model, df_train, df_val, LR, EPOCHS, adam_epsilon)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifi

Epochs: 1 | Train Loss:  0.017                 | Train Accuracy:  0.772                 | Val Loss:  0.014                 | Val Accuracy:  0.789


100%|██████████| 106/106 [00:33<00:00,  3.19it/s]


Epochs: 2 | Train Loss:  0.012                 | Train Accuracy:  0.855                 | Val Loss:  0.013                 | Val Accuracy:  0.812


100%|██████████| 106/106 [00:33<00:00,  3.19it/s]


Epochs: 3 | Train Loss:  0.010                 | Train Accuracy:  0.885                 | Val Loss:  0.012                 | Val Accuracy:  0.822


100%|██████████| 106/106 [00:33<00:00,  3.17it/s]


Epochs: 4 | Train Loss:  0.008                 | Train Accuracy:  0.907                 | Val Loss:  0.012                 | Val Accuracy:  0.829


100%|██████████| 106/106 [00:33<00:00,  3.20it/s]


Epochs: 5 | Train Loss:  0.007                 | Train Accuracy:  0.920                 | Val Loss:  0.012                 | Val Accuracy:  0.829


In [17]:
predictions = []

In [18]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_input['attention_mask'].to(device)
            input_id = test_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            predictions.append({'Actual':test_label,'Pred':output.argmax(dim=1)})
            acc = (output.argmax(dim=1) == test_label).sum().item()
            total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    return predictions
    
predictions = evaluate(model, df_test)

Test Accuracy:  0.855


In [19]:
df_metric = pd.DataFrame()

actuals = []
preds = []
for res in predictions:
  act = res['Actual']
  pre = res['Pred']
  actuals.extend(act.tolist())
  preds.extend(pre.tolist())

print(classification_report(actuals, preds))

              precision    recall  f1-score   support

           0       0.90      0.82      0.86       225
           1       0.81      0.89      0.85       197

    accuracy                           0.86       422
   macro avg       0.86      0.86      0.86       422
weighted avg       0.86      0.86      0.86       422



In [20]:
output_model = '../data/distilbert_base_uncased_empathy_classifier.pth'

In [22]:
torch.save({'model_state_dict': model.state_dict()}, output_model)